In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [153]:
df = pd.read_csv('Data.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [154]:
import os
def clean_data(df):
    os.makedirs('pipelines', exist_ok=True)
    df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)
    # Label encoding
    le = LabelEncoder()
    df['Gender'] = le.fit_transform(df['Gender'])
    with open('pipelines/label_encoder.pkl','wb') as f:
        pickle.dump(le, f)   # save encoder, not data
    # One-hot encoding
    one = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    geo_encoded = one.fit_transform(df[['Geography']])
    with open('pipelines/one_hot_encoding.pkl','wb') as f:
        pickle.dump(one, f)  # save encoder, not data
    
    # Convert back to DataFrame with column names
    geo_encoded_df = pd.DataFrame(
        geo_encoded,
        columns=one.get_feature_names_out(['Geography']),
    )
    # Drop original and add encoded
    df = pd.concat([df.drop(['Geography'], axis=1), geo_encoded_df], axis=1)
    return df

In [155]:
df1 = clean_data(df)

In [156]:
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [157]:
df1.describe()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.545700,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.501400,0.250900,0.247700
std,96.653299,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.500023,0.433553,0.431698
min,350.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,1.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,1.000000,0.000000,0.000000
75%,718.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,1.000000,0.000000
max,850.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


In [158]:
from sklearn.preprocessing import StandardScaler as ss
s = ss()
X = df1.drop(['Exited'],axis=1)
X = s.fit_transform(X)
y = df1.Exited
with open('pipelines/scaler.pkl','wb') as f:
    pickle.dump(s,f)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## ANN Implemntation

In [165]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam

In [166]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    # Dense(16, activation='relu'),
    # Dense(8, activation='relu'),
    Dense(1,activation='sigmoid')])

d:\Courses\Analytyx Camp\Data-Science-Batch-02\Complete Python\Lecture 19+20\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [167]:
model.compile(optimizer=Adam(learning_rate=0.001),
                loss = 'binary_crossentropy',
                metrics = ['accuracy','mse','mae'])

In [168]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tensorboard = TensorBoard(log_dir="logs")

In [169]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=32,callbacks=[early_stopping,tensorboard])

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7958 - loss: 0.4583 - mae: 0.3034 - mse: 0.1474 - val_accuracy: 0.8300 - val_loss: 0.3964 - val_mae: 0.2628 - val_mse: 0.1238
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8349 - loss: 0.3912 - mae: 0.2529 - mse: 0.1215 - val_accuracy: 0.8485 - val_loss: 0.3612 - val_mae: 0.2390 - val_mse: 0.1102
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8514 - loss: 0.3604 - mae: 0.2293 - mse: 0.1105 - val_accuracy: 0.8585 - val_loss: 0.3494 - val_mae: 0.2273 - val_mse: 0.1065
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8562 - loss: 0.3463 - mae: 0.2187 - mse: 0.1060 - val_accuracy: 0.8560 - val_loss: 0.3457 - val_mae: 0.2123 - val_mse: 0.1052
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8593 - loss: 0.3413 - mae: 0.2122 - mse: 0.1045 - val_accuracy: 0.8540 - val_loss: 0.3431 - val_mae: 0.1985 - val_mse: 0.1046
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms

In [170]:
os.makedirs('models',exist_ok=True)
model.save('models/model.h5')